Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [3]:
pickle_file = './notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

### 两层卷积，未池化

In [14]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [15]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.277067
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 0.944812
Minibatch accuracy: 62.5%
Validation accuracy: 50.5%
Minibatch loss at step 100: 1.059388
Minibatch accuracy: 56.2%
Validation accuracy: 73.5%
Minibatch loss at step 150: 1.150371
Minibatch accuracy: 62.5%
Validation accuracy: 72.0%
Minibatch loss at step 200: 1.054395
Minibatch accuracy: 81.2%
Validation accuracy: 78.0%
Minibatch loss at step 250: 0.822889
Minibatch accuracy: 75.0%
Validation accuracy: 78.1%
Minibatch loss at step 300: 0.907918
Minibatch accuracy: 68.8%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.350143
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.695514
Minibatch accuracy: 87.5%
Validation accuracy: 80.1%
Minibatch loss at step 450: 1.028287
Minibatch accuracy: 68.8%
Validation accuracy: 78.8%
Minibatch loss at step 500: 0.632704
Minibatch accuracy: 81.2%
Validation accuracy: 81.6%
Mi

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [61]:
import math

print(7/2)
print(math.ceil(math.ceil(image_size//2/2)//2/2)**2*depth)

3.5
64


### 两层卷积+两次pooling

In [50]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [floor(floor(image_size//2/2)//2/2)**2*deepth , num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def max_pool_2x2(x):
    """
    padding = 'SAME': Round down (only full size windows are considered).
    padding = 'VALID': Round up (partial windows are included).
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')

  # Model.
  def model(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    max_pool = max_pool_2x2(hidden)
    conv = tf.nn.conv2d(max_pool, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    max_pool = max_pool_2x2(hidden)
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [51]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.245908
Minibatch accuracy: 18.8%
Validation accuracy: 10.2%
Minibatch loss at step 50: 1.209020
Minibatch accuracy: 81.2%
Validation accuracy: 35.3%
Minibatch loss at step 100: 1.081415
Minibatch accuracy: 62.5%
Validation accuracy: 65.3%
Minibatch loss at step 150: 1.404632
Minibatch accuracy: 68.8%
Validation accuracy: 65.5%
Minibatch loss at step 200: 1.074799
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 250: 0.892156
Minibatch accuracy: 68.8%
Validation accuracy: 73.6%
Minibatch loss at step 300: 1.045174
Minibatch accuracy: 62.5%
Validation accuracy: 80.0%
Minibatch loss at step 350: 0.463666
Minibatch accuracy: 87.5%
Validation accuracy: 81.0%
Minibatch loss at step 400: 0.827460
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 450: 1.030784
Minibatch accuracy: 68.8%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.843199
Minibatch accuracy: 75.0%
Validation accuracy: 82.4%
M

Validation accuracy: 88.2%
Minibatch loss at step 4550: 0.425939
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 4600: 0.536162
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 4650: 0.422463
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 4700: 0.642050
Minibatch accuracy: 75.0%
Validation accuracy: 88.3%
Minibatch loss at step 4750: 0.403506
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 4800: 0.342163
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 4850: 0.621787
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 4900: 0.085086
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 4950: 1.073914
Minibatch accuracy: 62.5%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 0.901499
Minibatch accuracy: 75.0%
Validation accuracy: 87.8%
Test accuracy: 93.5%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### 两层卷积+两次pooling
- learning rate decay
- dropout (本次实验在对valid set和test set进行预测时也进行了dropout，导致结果低于预期)

In [30]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
keep_prob = 0.5
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  layer3_weights = tf.Variable(tf.truncated_normal(
      [math.ceil(math.ceil(image_size//2/2)//2/2)**2*depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='SAME')

  # Model.
  def model(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    max_pool = max_pool_2x2(hidden)
    conv = tf.nn.conv2d(max_pool, layer2_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    max_pool = max_pool_2x2(hidden)
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), keep_prob=keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  """
  decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
  """
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.7, staircase=True, name=None)
  optimizer = tf.train.MomentumOptimizer(learning_rate, 0.6).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [48]:
num_steps = 30001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.235619
Minibatch accuracy: 31.2%
Validation accuracy: 9.8%
Minibatch loss at step 50: 2.284733
Minibatch accuracy: 6.2%
Validation accuracy: 16.4%
Minibatch loss at step 100: 1.961445
Minibatch accuracy: 31.2%
Validation accuracy: 24.2%
Minibatch loss at step 150: 1.859371
Minibatch accuracy: 62.5%
Validation accuracy: 41.7%
Minibatch loss at step 200: 2.233802
Minibatch accuracy: 50.0%
Validation accuracy: 54.5%
Minibatch loss at step 250: 1.622989
Minibatch accuracy: 43.8%
Validation accuracy: 64.4%
Minibatch loss at step 300: 1.368005
Minibatch accuracy: 68.8%
Validation accuracy: 68.4%
Minibatch loss at step 350: 0.610750
Minibatch accuracy: 75.0%
Validation accuracy: 71.1%
Minibatch loss at step 400: 0.944678
Minibatch accuracy: 62.5%
Validation accuracy: 72.1%
Minibatch loss at step 450: 1.260144
Minibatch accuracy: 56.2%
Validation accuracy: 70.1%
Minibatch loss at step 500: 1.256263
Minibatch accuracy: 50.0%
Validation accuracy: 73.0%
Min

Validation accuracy: 83.2%
Minibatch loss at step 4550: 0.625353
Minibatch accuracy: 81.2%
Validation accuracy: 83.8%
Minibatch loss at step 4600: 0.618900
Minibatch accuracy: 68.8%
Validation accuracy: 83.7%
Minibatch loss at step 4650: 0.440837
Minibatch accuracy: 81.2%
Validation accuracy: 84.0%
Minibatch loss at step 4700: 0.777104
Minibatch accuracy: 81.2%
Validation accuracy: 84.1%
Minibatch loss at step 4750: 0.401248
Minibatch accuracy: 75.0%
Validation accuracy: 84.1%
Minibatch loss at step 4800: 0.316511
Minibatch accuracy: 87.5%
Validation accuracy: 84.1%
Minibatch loss at step 4850: 1.213853
Minibatch accuracy: 62.5%
Validation accuracy: 82.6%
Minibatch loss at step 4900: 0.226186
Minibatch accuracy: 93.8%
Validation accuracy: 83.8%
Minibatch loss at step 4950: 1.406403
Minibatch accuracy: 75.0%
Validation accuracy: 83.0%
Minibatch loss at step 5000: 1.080698
Minibatch accuracy: 68.8%
Validation accuracy: 82.9%
Minibatch loss at step 5050: 0.503354
Minibatch accuracy: 87.5%

Validation accuracy: 85.3%
Minibatch loss at step 9050: 0.656280
Minibatch accuracy: 75.0%
Validation accuracy: 84.9%
Minibatch loss at step 9100: 0.509453
Minibatch accuracy: 81.2%
Validation accuracy: 85.4%
Minibatch loss at step 9150: 0.105551
Minibatch accuracy: 100.0%
Validation accuracy: 84.9%
Minibatch loss at step 9200: 0.354775
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 9250: 0.422752
Minibatch accuracy: 81.2%
Validation accuracy: 84.9%
Minibatch loss at step 9300: 0.196326
Minibatch accuracy: 93.8%
Validation accuracy: 84.7%
Minibatch loss at step 9350: 0.273613
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 9400: 0.441600
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 9450: 0.194682
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 9500: 0.076287
Minibatch accuracy: 100.0%
Validation accuracy: 84.4%
Minibatch loss at step 9550: 0.552548
Minibatch accuracy: 81.

Minibatch loss at step 13500: 0.879190
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 13550: 0.018949
Minibatch accuracy: 100.0%
Validation accuracy: 85.0%
Minibatch loss at step 13600: 0.275998
Minibatch accuracy: 93.8%
Validation accuracy: 85.4%
Minibatch loss at step 13650: 0.459190
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 13700: 0.178370
Minibatch accuracy: 100.0%
Validation accuracy: 86.2%
Minibatch loss at step 13750: 0.548113
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 13800: 0.445429
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 13850: 0.410725
Minibatch accuracy: 81.2%
Validation accuracy: 85.3%
Minibatch loss at step 13900: 0.378451
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 13950: 0.354918
Minibatch accuracy: 87.5%
Validation accuracy: 84.7%
Minibatch loss at step 14000: 0.203081
Minibatch accuracy: 93.8%
Validation ac

Minibatch loss at step 17950: 0.970277
Minibatch accuracy: 62.5%
Validation accuracy: 85.0%
Minibatch loss at step 18000: 0.465509
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 18050: 0.400290
Minibatch accuracy: 81.2%
Validation accuracy: 85.8%
Minibatch loss at step 18100: 0.418137
Minibatch accuracy: 93.8%
Validation accuracy: 85.8%
Minibatch loss at step 18150: 1.555319
Minibatch accuracy: 75.0%
Validation accuracy: 85.2%
Minibatch loss at step 18200: 0.556370
Minibatch accuracy: 81.2%
Validation accuracy: 84.5%
Minibatch loss at step 18250: 0.231180
Minibatch accuracy: 100.0%
Validation accuracy: 85.6%
Minibatch loss at step 18300: 0.399547
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 18350: 0.303111
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Minibatch loss at step 18400: 0.503002
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 18450: 0.589072
Minibatch accuracy: 81.2%
Validation acc

Minibatch loss at step 22400: 0.036640
Minibatch accuracy: 100.0%
Validation accuracy: 86.1%
Minibatch loss at step 22450: 0.223064
Minibatch accuracy: 93.8%
Validation accuracy: 85.2%
Minibatch loss at step 22500: 0.391017
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 22550: 0.157726
Minibatch accuracy: 93.8%
Validation accuracy: 85.2%
Minibatch loss at step 22600: 0.277832
Minibatch accuracy: 93.8%
Validation accuracy: 84.9%
Minibatch loss at step 22650: 0.365843
Minibatch accuracy: 87.5%
Validation accuracy: 85.4%
Minibatch loss at step 22700: 0.968999
Minibatch accuracy: 75.0%
Validation accuracy: 85.1%
Minibatch loss at step 22750: 0.250326
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 22800: 0.398456
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 22850: 0.329451
Minibatch accuracy: 87.5%
Validation accuracy: 86.2%
Minibatch loss at step 22900: 0.164185
Minibatch accuracy: 93.8%
Validation acc

Minibatch loss at step 26850: 0.261990
Minibatch accuracy: 93.8%
Validation accuracy: 86.3%
Minibatch loss at step 26900: 0.242819
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 26950: 0.624164
Minibatch accuracy: 81.2%
Validation accuracy: 85.2%
Minibatch loss at step 27000: 0.432986
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 27050: 0.400469
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 27100: 0.230709
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 27150: 0.346738
Minibatch accuracy: 87.5%
Validation accuracy: 85.5%
Minibatch loss at step 27200: 1.252655
Minibatch accuracy: 75.0%
Validation accuracy: 85.1%
Minibatch loss at step 27250: 1.256744
Minibatch accuracy: 62.5%
Validation accuracy: 85.5%
Minibatch loss at step 27300: 0.368849
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 27350: 0.677890
Minibatch accuracy: 75.0%
Validation accu

### 两层卷积+两次pooling¶
- learning rate decay
- dropout (本次实验在对valid set和test set进行预测时,去掉dropout了（理应如此）)

In [96]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
keep_prob = 0.8
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  new_size = math.ceil((((math.ceil(((image_size-patch_size+1)-2+1)/2)-patch_size+1)) - 2 + 1) / 2)
  print(new_size)
  layer3_weights = tf.Variable(tf.truncated_normal(
      [new_size**2*depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='VALID')

  # Model.
  def model(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    # input 28 * 28
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer1_biases)
    # 28 - 5(patch_size) + 1 = 24, input 24 * 24
    max_pool = max_pool_2x2(hidden)
    # ceil((24 - 2 + 1) / 2) = 12, input 12 * 12 
    conv = tf.nn.conv2d(max_pool, layer2_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 12 - 5 + 1 = 8, input 8 * 8
    max_pool = max_pool_2x2(hidden)
    # ceil((8 - 2 + 1) / 2) = 4, input 4 * 4 
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), keep_prob=keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  """
  decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
  """
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.37, staircase=True, name=None)
  optimizer = tf.train.MomentumOptimizer(learning_rate, 0.6).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

4


In [97]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.759915
Minibatch accuracy: 18.8%
Validation accuracy: 10.2%
Minibatch loss at step 50: 0.980603
Minibatch accuracy: 75.0%
Validation accuracy: 44.3%
Minibatch loss at step 100: 1.773895
Minibatch accuracy: 37.5%
Validation accuracy: 61.5%
Minibatch loss at step 150: 1.408789
Minibatch accuracy: 50.0%
Validation accuracy: 69.8%
Minibatch loss at step 200: 1.232012
Minibatch accuracy: 75.0%
Validation accuracy: 74.8%
Minibatch loss at step 250: 1.298495
Minibatch accuracy: 68.8%
Validation accuracy: 74.0%
Minibatch loss at step 300: 1.401020
Minibatch accuracy: 62.5%
Validation accuracy: 76.3%
Minibatch loss at step 350: 0.533752
Minibatch accuracy: 81.2%
Validation accuracy: 78.7%
Minibatch loss at step 400: 1.027910
Minibatch accuracy: 62.5%
Validation accuracy: 79.3%
Minibatch loss at step 450: 1.132483
Minibatch accuracy: 68.8%
Validation accuracy: 75.7%
Minibatch loss at step 500: 0.957476
Minibatch accuracy: 62.5%
Validation accuracy: 79.0%
M

Validation accuracy: 84.8%
Minibatch loss at step 4550: 0.318774
Minibatch accuracy: 93.8%
Validation accuracy: 86.2%
Minibatch loss at step 4600: 0.576124
Minibatch accuracy: 81.2%
Validation accuracy: 85.7%
Minibatch loss at step 4650: 0.838885
Minibatch accuracy: 81.2%
Validation accuracy: 84.4%
Minibatch loss at step 4700: 0.677872
Minibatch accuracy: 81.2%
Validation accuracy: 85.9%
Minibatch loss at step 4750: 0.515390
Minibatch accuracy: 81.2%
Validation accuracy: 86.4%
Minibatch loss at step 4800: 0.125188
Minibatch accuracy: 93.8%
Validation accuracy: 86.1%
Minibatch loss at step 4850: 0.782236
Minibatch accuracy: 75.0%
Validation accuracy: 85.2%
Minibatch loss at step 4900: 0.304234
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 4950: 1.137106
Minibatch accuracy: 56.2%
Validation accuracy: 85.7%
Minibatch loss at step 5000: 0.708375
Minibatch accuracy: 75.0%
Validation accuracy: 85.8%
Minibatch loss at step 5050: 0.555777
Minibatch accuracy: 81.2%

Validation accuracy: 87.2%
Minibatch loss at step 9050: 0.401951
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 9100: 0.279899
Minibatch accuracy: 93.8%
Validation accuracy: 86.5%
Minibatch loss at step 9150: 0.011444
Minibatch accuracy: 100.0%
Validation accuracy: 87.1%
Minibatch loss at step 9200: 0.514852
Minibatch accuracy: 87.5%
Validation accuracy: 86.6%
Minibatch loss at step 9250: 0.436558
Minibatch accuracy: 87.5%
Validation accuracy: 87.0%
Minibatch loss at step 9300: 0.267190
Minibatch accuracy: 93.8%
Validation accuracy: 87.2%
Minibatch loss at step 9350: 0.324063
Minibatch accuracy: 93.8%
Validation accuracy: 86.4%
Minibatch loss at step 9400: 0.339728
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 9450: 0.126341
Minibatch accuracy: 100.0%
Validation accuracy: 87.1%
Minibatch loss at step 9500: 0.149018
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 9550: 0.696271
Minibatch accuracy: 87.

Minibatch loss at step 13500: 0.269771
Minibatch accuracy: 93.8%
Validation accuracy: 86.9%
Minibatch loss at step 13550: 0.024404
Minibatch accuracy: 100.0%
Validation accuracy: 87.7%
Minibatch loss at step 13600: 0.288799
Minibatch accuracy: 93.8%
Validation accuracy: 87.1%
Minibatch loss at step 13650: 0.376066
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 13700: 0.128913
Minibatch accuracy: 100.0%
Validation accuracy: 87.0%
Minibatch loss at step 13750: 0.393597
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 13800: 0.348619
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 13850: 0.411365
Minibatch accuracy: 87.5%
Validation accuracy: 86.4%
Minibatch loss at step 13900: 0.307156
Minibatch accuracy: 87.5%
Validation accuracy: 88.1%
Minibatch loss at step 13950: 0.343587
Minibatch accuracy: 87.5%
Validation accuracy: 86.3%
Minibatch loss at step 14000: 0.236785
Minibatch accuracy: 87.5%
Validation ac

Minibatch loss at step 17950: 0.474897
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
Minibatch loss at step 18000: 0.408802
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 18050: 0.339443
Minibatch accuracy: 93.8%
Validation accuracy: 87.6%
Minibatch loss at step 18100: 0.545194
Minibatch accuracy: 93.8%
Validation accuracy: 88.3%
Minibatch loss at step 18150: 0.899432
Minibatch accuracy: 75.0%
Validation accuracy: 87.8%
Minibatch loss at step 18200: 0.473348
Minibatch accuracy: 81.2%
Validation accuracy: 88.0%
Minibatch loss at step 18250: 0.597344
Minibatch accuracy: 81.2%
Validation accuracy: 86.9%
Minibatch loss at step 18300: 0.297456
Minibatch accuracy: 87.5%
Validation accuracy: 88.0%
Minibatch loss at step 18350: 0.712859
Minibatch accuracy: 75.0%
Validation accuracy: 87.4%
Minibatch loss at step 18400: 0.419558
Minibatch accuracy: 81.2%
Validation accuracy: 87.8%
Minibatch loss at step 18450: 0.594173
Minibatch accuracy: 75.0%
Validation accu

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64
keep_prob = 0.8
graph = tf.Graph()
import math
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  
  new_size = math.ceil((((math.ceil(((image_size-patch_size+1)-2+1)/2)-patch_size+1)) - 2 + 1) / 2)
  print(new_size)
  layer3_weights = tf.Variable(tf.truncated_normal(
      [new_size**2*depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1], padding='VALID')

  # Model.
  def model(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    # input 28 * 28
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer1_biases)
    # 28 - 5(patch_size) + 1 = 24, input 24 * 24
    max_pool = max_pool_2x2(hidden)
    # ceil((24 - 2 + 1) / 2) = 12, input 12 * 12 
    conv = tf.nn.conv2d(max_pool, layer2_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 12 - 5 + 1 = 8, input 8 * 8
    max_pool = max_pool_2x2(hidden)
    # ceil((8 - 2 + 1) / 2) = 4, input 4 * 4 
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.dropout(tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases), keep_prob=keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

  def predict(data):
    # strides: A list of ints. 1-D of length 4. The stride of the sliding window for each dimension of input
    # input 28 * 28
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer1_biases)
    # 28 - 5(patch_size) + 1 = 24, input 24 * 24
    max_pool = max_pool_2x2(hidden)
    # ceil((24 - 2 + 1) / 2) = 12, input 12 * 12 
    conv = tf.nn.conv2d(max_pool, layer2_weights, strides=[1, 1, 1, 1], padding='VALID')
    hidden = tf.nn.relu(conv + layer2_biases)
    # 12 - 5 + 1 = 8, input 8 * 8
    max_pool = max_pool_2x2(hidden)
    # ceil((8 - 2 + 1) / 2) = 4, input 4 * 4 
    shape = max_pool.get_shape().as_list()
    reshape = tf.reshape(max_pool, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  """
  decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)
  """
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 100, 0.37, staircase=True, name=None)
  optimizer = tf.train.MomentumOptimizer(learning_rate, 0.6).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(predict(tf_valid_dataset))
  test_prediction = tf.nn.softmax(predict(tf_test_dataset))

4


In [8]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.803486
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 0.927324
Minibatch accuracy: 87.5%
Validation accuracy: 61.6%
Minibatch loss at step 100: 1.348211
Minibatch accuracy: 43.8%
Validation accuracy: 68.2%
Minibatch loss at step 150: 1.423715
Minibatch accuracy: 68.8%
Validation accuracy: 73.3%
Minibatch loss at step 200: 1.243949
Minibatch accuracy: 68.8%
Validation accuracy: 73.4%
Minibatch loss at step 250: 1.026381
Minibatch accuracy: 56.2%
Validation accuracy: 79.0%
Minibatch loss at step 300: 0.820717
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 350: 0.253694
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Minibatch loss at step 400: 0.599398
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 450: 0.949975
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 500: 0.699220
Minibatch accuracy: 75.0%
Validation accuracy: 82.5%
M

Validation accuracy: 86.1%
Minibatch loss at step 4550: 0.440966
Minibatch accuracy: 87.5%
Validation accuracy: 87.2%
Minibatch loss at step 4600: 0.661960
Minibatch accuracy: 81.2%
Validation accuracy: 86.6%
Minibatch loss at step 4650: 0.548963
Minibatch accuracy: 87.5%
Validation accuracy: 86.7%
Minibatch loss at step 4700: 0.519950
Minibatch accuracy: 75.0%
Validation accuracy: 87.7%
Minibatch loss at step 4750: 0.556362
Minibatch accuracy: 81.2%
Validation accuracy: 87.1%
Minibatch loss at step 4800: 0.256407
Minibatch accuracy: 93.8%
Validation accuracy: 87.5%
Minibatch loss at step 4850: 0.654070
Minibatch accuracy: 75.0%
Validation accuracy: 87.5%
Minibatch loss at step 4900: 0.146518
Minibatch accuracy: 93.8%
Validation accuracy: 87.3%
Minibatch loss at step 4950: 1.175735
Minibatch accuracy: 62.5%
Validation accuracy: 86.9%
Minibatch loss at step 5000: 0.901807
Minibatch accuracy: 68.8%
Validation accuracy: 85.4%
Minibatch loss at step 5050: 0.654801
Minibatch accuracy: 75.0%

Validation accuracy: 87.2%
Minibatch loss at step 9050: 0.287797
Minibatch accuracy: 100.0%
Validation accuracy: 88.0%
Minibatch loss at step 9100: 0.167920
Minibatch accuracy: 93.8%
Validation accuracy: 88.0%
Minibatch loss at step 9150: 0.039274
Minibatch accuracy: 100.0%
Validation accuracy: 87.8%
Minibatch loss at step 9200: 0.370156
Minibatch accuracy: 87.5%
Validation accuracy: 87.5%
Minibatch loss at step 9250: 0.386327
Minibatch accuracy: 75.0%
Validation accuracy: 87.8%
Minibatch loss at step 9300: 0.243265
Minibatch accuracy: 87.5%
Validation accuracy: 87.6%
Minibatch loss at step 9350: 0.083319
Minibatch accuracy: 100.0%
Validation accuracy: 87.9%
Minibatch loss at step 9400: 0.274899
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 9450: 0.162802
Minibatch accuracy: 93.8%
Validation accuracy: 87.7%
Minibatch loss at step 9500: 0.128826
Minibatch accuracy: 93.8%
Validation accuracy: 87.8%
Minibatch loss at step 9550: 0.243911
Minibatch accuracy: 93

Minibatch loss at step 13500: 0.191527
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 13550: 0.007345
Minibatch accuracy: 100.0%
Validation accuracy: 88.3%
Minibatch loss at step 13600: 0.338702
Minibatch accuracy: 87.5%
Validation accuracy: 87.9%
Minibatch loss at step 13650: 0.220080
Minibatch accuracy: 93.8%
Validation accuracy: 88.4%
Minibatch loss at step 13700: 0.066398
Minibatch accuracy: 100.0%
Validation accuracy: 88.4%
Minibatch loss at step 13750: 0.396385
Minibatch accuracy: 87.5%
Validation accuracy: 88.7%
Minibatch loss at step 13800: 0.179199
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 13850: 0.374941
Minibatch accuracy: 87.5%
Validation accuracy: 88.3%
Minibatch loss at step 13900: 0.356554
Minibatch accuracy: 87.5%
Validation accuracy: 88.2%
Minibatch loss at step 13950: 0.487772
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 14000: 0.409940
Minibatch accuracy: 93.8%
Validation ac

Minibatch loss at step 17950: 0.762857
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 18000: 0.405371
Minibatch accuracy: 87.5%
Validation accuracy: 88.5%
Minibatch loss at step 18050: 0.454330
Minibatch accuracy: 81.2%
Validation accuracy: 88.6%
Minibatch loss at step 18100: 0.405725
Minibatch accuracy: 93.8%
Validation accuracy: 88.6%
Minibatch loss at step 18150: 0.646574
Minibatch accuracy: 75.0%
Validation accuracy: 88.8%
Minibatch loss at step 18200: 0.491613
Minibatch accuracy: 81.2%
Validation accuracy: 88.4%
Minibatch loss at step 18250: 0.562298
Minibatch accuracy: 81.2%
Validation accuracy: 87.9%
Minibatch loss at step 18300: 0.222044
Minibatch accuracy: 93.8%
Validation accuracy: 88.5%
Minibatch loss at step 18350: 0.224997
Minibatch accuracy: 93.8%
Validation accuracy: 88.7%
Minibatch loss at step 18400: 0.354835
Minibatch accuracy: 87.5%
Validation accuracy: 89.2%
Minibatch loss at step 18450: 0.430899
Minibatch accuracy: 87.5%
Validation accu